## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

# Object Detection using TAO EfficientDet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train an EfficientDet-D0 model on COCO dataset
* Evaluate the trained model
* Run pruning and finetuning with the trained model
* Run inference with the trained model and visualize the result
* Export the trained model to a .etlt file for deployment to DeepStream

### Table of Contents
This notebook shows an example use case for instance segmentation using the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1)
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Generate tfrecords](#head-3)
3. [Provide training specification](#head-4)
4. [Run TAO training](#head-5)
5. [Evaluate trained models](#head-6)
6. [Prune trained model](#head-7)
7. [Retrain pruned models](#head-8)
8. [Evaluate retrained model](#head-9)
9. [Visualize inferences](#head-10)

## 0. Set up env variables and set FIXME parameters <a class="anchor" id="head-0"></a>

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

#### FIXME
1. NUM_GPUS - set this to <= number of GPU's availble on the instance
1. EXPERIMENT_DIR - set this path to a folder location where pretrained models, checkpoints and log files during different model actions will be saved
1. delete_existing_experiments - set to True to remove existing pretrained models, checkpoints and log files of a previous experiment
1. DATA_DIR - set this path to a folder location where you want to dataset to be present
1. delete_existing_data - set this to True to remove existing preprocessed and original data

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
#FIXME1
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")

#FIXME2
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/efficientdet
#FIXME3
delete_existing_experiments = True
#FIXME4
%env DATA_DIR=/content/drive/MyDrive/coco_data/
#FIXME5
delete_existing_data = False

if delete_existing_experiments:
    !sudo rm -rf $EXPERIMENT_DIR
if delete_existing_data:
    !sudo rm -rf $DATA_DIR

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/efficientdet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

!sudo mkdir -p $DATA_DIR && sudo chmod -R 777 $DATA_DIR
!sudo mkdir -p $EXPERIMENT_DIR && sudo chmod -R 777 $EXPERIMENT_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

 We will be using the COCO dataset for the tutorial. The following script will download COCO dataset automatically and convert it to TFRecords. 

In [ ]:
# Download and preprocess data
!bash $SPECS_DIR/download_coco.sh $DATA_DIR

Note that the dataset conversion scripts provided in `specs` are intended for the standard COCO dataset. If your data doesn't have `caption` groundtruth or test set, you can modify `download_and_preprocess_coco.sh` and `create_coco_tf_record.py` by commenting out corresponding variables.

In [ ]:
# verify
!ls -l $DATA_DIR

### Download pretrained model from NGC

 We will use NGC CLI to get the pre-trained models. For more details, go to ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && sudo chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/pretrained_efficientdet:efficientnet_b0*

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_efficientdet:efficientnet_b0 --dest $EXPERIMENT_DIR

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $EXPERIMENT_DIR/pretrained_efficientdet_vefficientnet_b0

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
#FIXME
%env GENERAL_WHL_PATH=/content/drive/MyDrive/tf/general_whl
#FIXME
%env CODEBASE_WHL_PATH=/content/drive/MyDrive/tf/codebase_whl

import os
if os.path.exists(os.environ["GENERAL_WHL_PATH"]) and os.path.exists(os.environ["GENERAL_WHL_PATH"]):
    if os.environ["GOOGLE_COLAB"] == "1":
        os.environ["bash_script"] = "setup_env.sh"
    else:
        os.environ["bash_script"] = "setup_env_desktop.sh"

    !sed -i "s|PATH_TO_GENERAL_WHL|$GENERAL_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_CODEBASE_WHL|$CODEBASE_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
    !sed -i "s|PATH_TO_COLAB_NOTEBOOKS|$COLAB_NOTEBOOKS_PATH|g" $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script

    !sh $COLAB_NOTEBOOKS_PATH/tensorflow/$bash_script
else:
    raise("Error, enter the whl paths correctly")

In [ ]:
if os.environ.get("PYTHONPATH","") == "":
    os.environ["PYTHONPATH"] = ""
os.environ["PYTHONPATH"]+=":/opt/nvidia/"
if os.environ["GOOGLE_COLAB"] == "1":
    os.environ["PYTHONPATH"]+=":/usr/local/lib/python3.6/dist-packages/third_party/nvml"
else:
    os.environ["PYTHONPATH"]+=":/home_duplicate/rarunachalam/miniconda3/envs/tf_py_36/lib/python3.6/site-packages/third_party/nvml" # FIX MINICONDA PATH

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env TAO_DOCKER_DISABLE=1

%env KEY=nvidia_tlt
%env NUM_GPUS=1

# Change the paths according to your directory structure, these are just examples
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks
if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
    raise("Error, enter the path of the colab notebooks repo correctly")
%env EXPERIMENT_DIR=/content/drive/MyDrive/results/efficientdet
%env DATA_DIR=/content/drive/MyDrive/coco_data/

SPECS_DIR=f"{os.environ['COLAB_NOTEBOOKS_PATH']}/tensorflow/efficientdet/specs"
%env SPECS_DIR={SPECS_DIR}
# Showing list of specification files.
!ls -rlt $SPECS_DIR

## 3. Generate tfrecords <a class="anchor" id="head-3"></a>
* Create the tfrecords on the dataset split

In [ ]:
# convert training data to TFRecords
!tao efficientdet_tf1 dataset_convert -i $DATA_DIR/raw-data/train2017 \
                                  -a $DATA_DIR/raw-data/annotations/instances_train2017.json \
                                  -o $DATA_DIR --include_masks -t train -s 256

In [ ]:
 # convert validation data to TFRecords
 !tao efficientdet_tf1 dataset_convert -i $DATA_DIR/raw-data/val2017 \
                                  -a $DATA_DIR/raw-data/annotations/instances_val2017.json \
                                  -o $DATA_DIR --include_masks -t val -s 32

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Tfrecords for the train datasets
    * In order to use the newly generated tfrecords, update the dataset_config parameter in the spec file at `$SPECS_DIR/efficientdet_d0_train.txt` 
Note that the learning rate in the spec file is set for 4 GPU training. If you have N gpus, you should divide LR by 4/N.
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* **Note that the sample spec is not meant to produce SOTA accuracy on COCO. To reproduce SOTA, you might want to use TAO to train an ImageNet model first and change the total_steps to 100K or above.**

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/efficientdet_d0_train.txt
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/efficientdet_d0_train.txt
!cat $SPECS_DIR/efficientdet_d0_train.txt

## 5. Train an Efficientdet model <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models
* Evaluation uses COCO metrics. For more info, please refer to: https://cocodataset.org/#detection-eval
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_unpruned

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao efficientdet_tf1 train -e $SPECS_DIR/efficientdet_d0_train.txt \
                        -d $EXPERIMENT_DIR/experiment_dir_unpruned\
                        -k $KEY \
                        --gpus $NUM_GPUS

In [ ]:
print("To resume training from a checkpoint, simply run the same training script. It will pick up from where it's left.")
# !tao efficientdet_tf1 train -e $SPECS_DIR/efficientdet_d0_train.txt \
#                         -d $EXPERIMENT_DIR/experiment_dir_unpruned\
#                         -k $KEY \
#                         --gpus $NUM_GPUS

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $EXPERIMENT_DIR/experiment_dir_unpruned/

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>

In [ ]:
%env NUM_STEP=1250

In [ ]:
!tao efficientdet_tf1 evaluate -e $SPECS_DIR/efficientdet_d0_train.txt \
                           -m $EXPERIMENT_DIR/experiment_dir_unpruned/model.step-$NUM_STEP.tlt \
                           -k $KEY

## 7. Prune <a class="anchor" id="head-7"></a>

- Specify pre-trained model
- Equalization criterion
- Threshold for pruning.
- A key to save and load the model
- Output directory to store the model

Usually, you just need to adjust -pth (threshold) for accuracy and model size trade off. Higher pth gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. 0.4 in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
# Create an output directory to save the pruned model.
!mkdir -p $EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao efficientdet_tf1 prune -m $EXPERIMENT_DIR/experiment_dir_unpruned/model.step-$NUM_STEP.tlt \
                        -o $EXPERIMENT_DIR/experiment_dir_pruned \
                        -pth 0.7 \
                        -k $KEY

In [ ]:
!ls -l $EXPERIMENT_DIR/experiment_dir_pruned

**Note** that you should retrain the pruned model first, as it cannot be directly used for evaluation or inference. 

## 8. Retrain pruned models <a class="anchor" id="head-8"></a>

- Model needs to be re-trained to bring back accuracy after pruning
- Specify re-training specification
- WARNING: training will take several hours or one day to complete

In [ ]:
!sed -i "s|TAO_DATA_PATH|$DATA_DIR/|g" $SPECS_DIR/efficientdet_d0_retrain.txt
!sed -i "s|EXPERIMENT_DIR_PATH|$EXPERIMENT_DIR/|g" $SPECS_DIR/efficientdet_d0_retrain.txt
!cat $SPECS_DIR/efficientdet_d0_retrain.txt

In [ ]:
!mkdir -p $EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
!tao efficientdet_tf1 train -e $SPECS_DIR/efficientdet_d0_retrain.txt \
                        -d $EXPERIMENT_DIR/experiment_dir_retrain\
                        -k $KEY \
                        --gpus $NUM_GPUS

## 9. Evaluate retrained model <a class="anchor" id="head-9"></a>

In [ ]:
%env NUM_STEP=1250

In [ ]:
!tao efficientdet_tf1 evaluate -e $SPECS_DIR/efficientdet_d0_retrain.txt \
                           -m $EXPERIMENT_DIR/experiment_dir_retrain/model.step-$NUM_STEP.tlt \
                           -k $KEY

## 10. Visualize inferences <a class="anchor" id="head-10"></a>
In this section, we run the `infer` tool to generate inferences on the trained models and visualize the results. The `infer` tool produces annotated image outputs. 

In [ ]:
# Running inference for detection on n images
!tao efficientdet_tf1 inference -i $DATA_DIR/raw-data/test2017 \
                            -o $EXPERIMENT_DIR/annotated_images \
                            -e $SPECS_DIR/efficientdet_d0_train.txt \
                            -m $EXPERIMENT_DIR/experiment_dir_unpruned/model.step-$NUM_STEP.tlt \
                            --label_map $SPECS_DIR/coco_labels.txt \
                            -k $KEY

In [ ]:
# Simple grid visualizer
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'annotated_images' # relative path from $EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)